In [2]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";
const history = new ChatMessageHistory();
await history.addMessage(new HumanMessage("hi"));
await history.addMessage(new AIMessage("What can I do for you?"));
const messages = await history.getMessages();
messages

[
  HumanMessage {
    "content": "hi",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  AIMessage {
    "content": "What can I do for you?",
    "additional_kwargs": {},
    "response_metadata": {},
    "tool_calls": [],
    "invalid_tool_calls": []
  }
]


```typescript
export abstract class BaseChatMessageHistory extends Serializable {
  public abstract getMessages(): Promise<BaseMessage[]>;

  public abstract addMessage(message: BaseMessage): Promise<void>;

  public abstract addUserMessage(message: string): Promise<void>;

  public abstract addAIChatMessage(message: string): Promise<void>;

  public abstract clear(): Promise<void>;
}
```

In [3]:
// 手动维护
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatDeepSeek } from "@langchain/deepseek";

const chatModel = new ChatDeepSeek({
    model: "deepseek-chat",
  });
const prompt = ChatPromptTemplate.fromMessages([
    ["system", `You are a helpful assistant. Answer all questions to the best of your ability.
    You are talkative and provides lots of specific details from its context. 
    If the you does not know the answer to a question, it truthfully says you do not know.`],
    new MessagesPlaceholder("history_message"),
]);

const chain = prompt.pipe(chatModel);


const history = new ChatMessageHistory();
await history.addMessage(new HumanMessage("hi, my name is Kai"));
const res1 = await chain.invoke({
    history_message: await history.getMessages()
})
res1



AIMessage {
  "id": "850eb88b-83fd-4c00-a178-e0ad6d4d23be",
  "content": "Hi Kai! It's great to meet you. How can I assist you today? Whether you have questions, need advice, or just want to chat, I'm here to help!",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 63,
      "completionTokens": 37,
      "totalTokens": 100
    },
    "finish_reason": "stop",
    "model_name": "deepseek-chat",
    "usage": {
      "prompt_tokens": 63,
      "completion_tokens": 37,
      "total_tokens": 100,
      "prompt_tokens_details": {
        "cached_tokens": 0
      },
      "prompt_cache_hit_tokens": 0,
      "prompt_cache_miss_tokens": 63
    },
    "system_fingerprint": "fp_3a5770e1b4_prod0225"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 37,
    "input_tokens": 63,
    "total_tokens": 100,
    "input_token_details": {
      "cache_read": 0
    },
    "output_token_details": {}
  }
}

In [4]:
// 这里把对话的结果也手动添加到历史记录中
await history.addMessage(res1)
await history.addMessage(new HumanMessage("What is my name?"));
const res2 = await chain.invoke({
    history_message: await history.getMessages()
})
res2

AIMessage {
  "id": "a9afee9a-b08d-464c-88e4-b0e29ed111bf",
  "content": "Your name is Kai! You introduced yourself earlier by saying, \"Hi, my name is Kai.\" If there's anything specific you'd like to discuss or ask, feel free to let me know! 😊",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 108,
      "completionTokens": 42,
      "totalTokens": 150
    },
    "finish_reason": "stop",
    "model_name": "deepseek-chat",
    "usage": {
      "prompt_tokens": 108,
      "completion_tokens": 42,
      "total_tokens": 150,
      "prompt_tokens_details": {
        "cached_tokens": 64
      },
      "prompt_cache_hit_tokens": 64,
      "prompt_cache_miss_tokens": 44
    },
    "system_fingerprint": "fp_3a5770e1b4_prod0225"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 42,
    "input_tokens": 108,
    "total_tokens": 150,
    "input_token_details": {
      "cache_read": 64
    },
    "output_tok

RunnableWithMessageHistory 有几个参数：
- runnable 就是需要被包裹的 chain，可以是任意 chain
- getMessageHistory 接收一个函数，函数需要根据传入的 _sessionId，去获取对应的 ChatMessageHistory 对象，这里我们没有 session 管理，所以就返回默认的对象
- inputMessagesKey 用户传入的信息 key 的名称，因为 RunnableWithMessageHistory 要自动记录用户和 llm 发送的信息，所以需要在这里声明用户以什么 key 传入信息
- historyMessagesKey，聊天记录在 prompt 中的 key，因为要自动的把聊天记录注入到 prompt 中。
outputMessagesKey，因为我们的 chain 只有一个输出就省略了，如果有多个输出需要指定哪个是 llm 的回复，也就是需要存储的信息。

In [5]:
// 自动维护chat history，由 RunnableWithMessageHistory 给任意 chain 包裹一层，就能添加聊天记录管理的能力
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
const chatModel = new ChatDeepSeek({
    model: "deepseek-chat",
  });
const prompt = ChatPromptTemplate.fromMessages([
    ["system", "You are a helpful assistant. Answer all questions to the best of your ability."],
    new MessagesPlaceholder("history_message"),
    ["human","{input}"]
]);

const history = new ChatMessageHistory();
const chain = prompt.pipe(chatModel)

const chainWithHistory = new RunnableWithMessageHistory({
  runnable: chain,
  getMessageHistory: (_sessionId) => history,
  inputMessagesKey: "input",
  historyMessagesKey: "history_message",
});
const res1 = await chainWithHistory.invoke({
    input: "hi, my name is Kai"
},{
    configurable: { sessionId: "none" }
})
res1

AIMessage {
  "id": "f465900d-63c9-485d-a9d1-0b604cc98173",
  "content": "Hi, Kai! Nice to meet you. How can I assist you today? 😊",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 25,
      "completionTokens": 18,
      "totalTokens": 43
    },
    "finish_reason": "stop",
    "model_name": "deepseek-chat",
    "usage": {
      "prompt_tokens": 25,
      "completion_tokens": 18,
      "total_tokens": 43,
      "prompt_tokens_details": {
        "cached_tokens": 0
      },
      "prompt_cache_hit_tokens": 0,
      "prompt_cache_miss_tokens": 25
    },
    "system_fingerprint": "fp_3a5770e1b4_prod0225"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 18,
    "input_tokens": 25,
    "total_tokens": 43,
    "input_token_details": {
      "cache_read": 0
    },
    "output_token_details": {}
  }
}

In [6]:
const res2 = await chainWithHistory.invoke({
    input: "我的名字叫什么？"
},{
    configurable: { sessionId: "none" }
})
res2

AIMessage {
  "id": "f4048d34-10dd-41c2-9e29-595e8d31918a",
  "content": "你的名字叫Kai。有什么我可以帮你的吗？😊",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 50,
      "completionTokens": 14,
      "totalTokens": 64
    },
    "finish_reason": "stop",
    "model_name": "deepseek-chat",
    "usage": {
      "prompt_tokens": 50,
      "completion_tokens": 14,
      "total_tokens": 64,
      "prompt_tokens_details": {
        "cached_tokens": 0
      },
      "prompt_cache_hit_tokens": 0,
      "prompt_cache_miss_tokens": 50
    },
    "system_fingerprint": "fp_3a5770e1b4_prod0225"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 14,
    "input_tokens": 50,
    "total_tokens": 64,
    "input_token_details": {
      "cache_read": 0
    },
    "output_token_details": {}
  }
}

In [7]:
await history.getMessages()

[
  HumanMessage {
    "content": "hi, my name is Kai",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  AIMessage {
    "id": "f465900d-63c9-485d-a9d1-0b604cc98173",
    "content": "Hi, Kai! Nice to meet you. How can I assist you today? 😊",
    "additional_kwargs": {},
    "response_metadata": {
      "tokenUsage": {
        "promptTokens": 25,
        "completionTokens": 18,
        "totalTokens": 43
      },
      "finish_reason": "stop",
      "model_name": "deepseek-chat",
      "usage": {
        "prompt_tokens": 25,
        "completion_tokens": 18,
        "total_tokens": 43,
        "prompt_tokens_details": {
          "cached_tokens": 0
        },
        "prompt_cache_hit_tokens": 0,
        "prompt_cache_miss_tokens": 25
      },
      "system_fingerprint": "fp_3a5770e1b4_prod0225"
    },
    "tool_calls": [],
    "invalid_tool_calls": [],
    "usage_metadata": {
      "output_tokens": 18,
      "input_tokens": 25,
      "total_tokens": 43,
      "input_token_

In [10]:
// RunnableWithMessageHistory 是将历史记录完整的传递到 llm中，我们可以对 llm 的历史记录进行更多操作，例如只传递最近的 k 条历史记录等。

// 下面是一个总结历史记录的chain接收两个参数，
// summary，上一次总结的信息
// new_lines，用户和 llm 新的回复
import {RunnableSequence} from "@langchain/core/runnables";
import {StringOutputParser } from "@langchain/core/output_parsers";

const summaryModel = new ChatDeepSeek({
    model:'deepseek-chat'
});
const summaryPrompt = ChatPromptTemplate.fromTemplate(`
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:
`); 

const summaryChain = RunnableSequence.from([
    summaryPrompt,
    summaryModel,
    new StringOutputParser(),
])

const newSummary = await summaryChain.invoke({
    summary:"",
    new_lines: "I'm 18"
})
await summaryChain.invoke({
    summary: newSummary,
    new_lines: "I'm male"
})

"The person mentioned that they are an 18-year-old male."

我们使用 new RunnablePassthrough({func: (input)=> void})，是有两个目的：

- 如果我们只写 new RunnablePassthrough()，那就是把用户输入的 input 再传递到下一个 runnable 节点中，不做任何操作。因为 RunnableMap 返回值是对其中每个 chain 执行，然后将返回值作为结果传递给下一个 runnable 节点，如果我们不对 input 使用 RunnablePassthrough 则下个节点就拿不到 input 的值
- new RunnablePassthrough({func: (input)=> void}) 中的 func 函数是在传递 input 的过程中，执行一个函数，这个函数返回值是 void，也就是无论其内容是什么，都不会对 input 造成影响。

In [12]:
import {RunnablePassthrough} from "@langchain/core/runnables";
import {getBufferString} from "langchain/memory";
const chatModel = new ChatDeepSeek({
    model:'deepseek-chat'
});
const chatPrompt = ChatPromptTemplate.fromMessages([
    ["system", `You are a helpful assistant. Answer all questions to the best of your ability.

    Here is the chat history summary:
    {history_summary}
    `],
    ["human","{input}"]
]);
let summary = ""
const history = new ChatMessageHistory();

const chatChain = RunnableSequence.from([
    {
        input: new RunnablePassthrough({
             func: (input) => history.addUserMessage(input)
        })
    },
    RunnablePassthrough.assign({
        history_summary: () => summary
    }),
    chatPrompt,
    chatModel,
    new StringOutputParser(),
    new RunnablePassthrough({
        func: async (input) => {
            history.addAIChatMessage(input)
            const messages = await history.getMessages()
            const new_lines = getBufferString(messages)
            const newSummary = await summaryChain.invoke({
                summary,
                new_lines
            })
            history.clear()
            summary = newSummary      
        }
    })
])



In [13]:
await chatChain.invoke("我现在饿了")

"你现在饿了的话，可以考虑以下几种选择：\n" +
  "\n" +
  "1. **自己做饭**：如果你有食材，可以做一些简单的饭菜，比如炒饭、面条、煎蛋等。\n" +
  "2. **点外卖**：如果你不想做饭，可以点外卖，选择你喜欢的餐厅和菜品。\n" +
  "3. **外出就餐**：如果你有时间，可以出去找一家餐厅吃饭，享受一下外面的美食。\n" +
  "4. **零食**：如果你只是有点饿，可以先吃点零食垫垫肚子，比如水果、坚果、饼干等。\n" +
  "\n" +
  "你更倾向于哪种选择呢？"

In [14]:
await chatChain.invoke("我今天想吃方便面")

"方便面确实是一个快速解决饥饿的好选择！你可以根据自己的口味选择不同的调味包，或者加入一些额外的食材来提升口感。比如，你可以加一个鸡蛋、一些蔬菜（如青菜、胡萝卜）、或者一些肉类（如火腿、鸡肉）。这样不仅能让方便面更美味，还能增加营养。\n" +
  "\n" +
  "如果你有时间和条件，可以尝试以下几种升级版方便面的做法：\n" +
  "\n" +
  "1. **鸡蛋方便面**：在煮面的时候打入一个鸡蛋，可以做成荷包蛋或者打散成蛋花。\n" +
  "2. **蔬菜方便面**：加入一些新鲜的蔬菜，比如菠菜、豆芽、蘑菇等，增加纤维和维生素。\n" +
  "3. **肉类方便面**：加入一些熟食肉类，如火腿、培根、鸡肉片等，增加蛋白质。\n" +
  "4. **芝士方便面**：在煮好的方便面上撒上一些芝士片，等芝士融化后搅拌均匀，味道会更浓郁。\n" +
  "\n" +
  "你平时喜欢怎么吃方便面呢？有没有特别喜欢的口味或搭配？"

In [16]:
summary

"The human mentions feeling hungry and expresses a craving for instant noodles. The AI acknowledges this as a quick solution and suggests enhancing the noodles by adding ingredients like eggs, vegetables, or meat to improve flavor and nutrition. The AI also provides specific ideas for upgrading instant noodles, such as adding eggs, vegetables, meat, or cheese, and asks the human about their preferred way of eating instant noodles, including any favorite flavors or combinations. \n" +
  "\n" +
  "Summary: The human craves instant noodles, and the AI recommends enhancing them with additional ingredients like eggs, vegetables, or meat for better taste and nutrition. The AI offers specific upgrade ideas and asks about the human's preferred instant noodle preparation and flavors."